In [1]:
import bempp.api, numpy as np, time, os, matplotlib.pyplot as plt
from math import pi
from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz

In [2]:
# This python must be saved in a directory where you have a folder named
# /Molecule/Molecule_Name, as obviusly Molecule_Name holds a .pdb text file
# ; otherwise, this function won't do anything.

# With the .pdb file we can build .pqr & .xyzr files, and they don't change when the mesh density is changed.
def pdb_to_pqr(mol_name , stern_thickness , method = 'amber'):
    
    path = os.getcwd()
        
    pdb_file , pdb_directory = mol_name+'.pdb' , os.path.join('Molecule',mol_name)
    pqr_file , xyzr_file     = mol_name+'.pqr' , mol_name+'.xyzr'
    
    # The apbs-pdb2pqr rutine, allow us to generate a .pqr file
    pdb2pqr_dir = os.path.join('Software','apbs-pdb2pqr-master','pdb2pqr','main.py')
    exe=('python2.7  ' + pdb2pqr_dir + ' '+ os.path.join(pdb_directory,pdb_file) +
         ' --ff='+method+' ' + os.path.join(pdb_directory,mol_name)   )
    
    os.system(exe)
    
    # Now, .pqr file contains unneeded text inputs, we will save the rows starting with 'ATOM'.
    
    pqr_Text = open( os.path.join(pdb_directory , pqr_file) ).read()
    pqr_Text_xyzr = open(os.path.join(pdb_directory , xyzr_file )  ,'w+')

    c=0
    for i in pqr_Text.split('\n'):
        row=i.split()
        if row[0]=='ATOM':
            aux=' '.join(map( str  , row[5:8])) + ' ' + str(row[-1])
            pqr_Text_xyzr.write(aux + '\n')   
    pqr_Text_xyzr.close()
    
    print('Global .pqr & .xyzr ready.')
    
    # The exterior interface is easy to add, by increasing each ath
    if stern_thickness>0: 
        xyzr_file_stern = pdb_directory + mol_name +'_stern.xyzr'
        pqr_Text_xyzr_s = open(xyzr_file_stern ,'w')
        
        for i in pqr_Text.split('\n'):
            row=i.split()
            if row[0]=='ATOM':
                R_vv=float(row[-1])+stern_thickness
                pqr_Text_xyzr_s.write(row[5]+' '+row[6]+' '+row[7]+' '+str(R_vv)+'\n' )      
        pqr_Text_xyzr_s.close()
        print('Global _stern.pqr & _stern.xyzr ready.')
    
    return 
    

def xyzr_to_msh(mol_name , dens , probe_radius , stern_thickness , min_area ):

    pdb_directory = os.path.join('Molecule',mol_name)
    xyzr_file     = os.path.join(pdb_directory, mol_name + '.xyzr') 
    if stern_thickness > 0:  xyzr_s_file = os.path.join(pdb_directory , mol_name + '_stern.xyzr'  )
    
    # The executable line must be:
    #  path/Software/msms/msms.x86_64Linux2.2.6.1 
    # -if path/mol_name.xyzr       (Input File)
    # -of path/mol_name -prob 1.4 -d 3.    (Output File)
   
    # The directory of msms needs to be checked, it must be saved in the same folder that is this file
    msms= os.path.join('Software','msms','msms.x86_64Linux2.2.6.1')
    
    mode= ' -no_header'
    prob_rad, dens_msh = ' -prob ' + str(probe_radius), ' -d ' + str(dens)
    exe= (msms
          +' -if ' + xyzr_file
          +' -of ' + os.path.join(pdb_directory , mol_name )
          + prob_rad  + dens_msh + mode )
    os.system(exe)
    print('Normal .vert & .face Done')
    
    # Again, the possibility of using a stern layer is available
    
            
    grid = factory_fun_msh( pdb_directory , mol_name , min_area )
    print('Normal .msh Done')
    
    if stern_thickness > 0:
        prob_rad, dens_msh = ' -prob ' + str(probe_radius), ' -d ' + str(dens)
        exe= (msms+' -if '  + xyzr_s_file + 
          ' -of ' + os.path.join(pdb_directory , mol_name +'_stern') + prob_rad  + dens_msh  + mode )
        os.system(exe)
        print('Stern .vert & .face Done')
        stern_grid= factory_fun_msh( pdb_directory , mol_name+'_stern', min_area )
        print('Stern .msh Done')
    
    print('Mesh Ready')
    
def factory_fun_msh( pdb_directory , mol_name , min_area):
    # Factory function for creating a .msh file from .vert & .face files
    
    factory = bempp.api.grid.GridFactory()
    
    vert_Text = open( os.path.join(pdb_directory , mol_name +'.vert' ) ).read().split('\n')
    face_Text = open( os.path.join(pdb_directory , mol_name +'.face' ) ).read().split('\n')
    

    # Counters for small triangles, and total ignored area
    xcount, atotal, a_excl = 0, 0., 0.
    vertex = np.empty((0,3))

    # Create the grid with the factory method
    factory = bempp.api.grid.GridFactory()
    for line in vert_Text:
        line = line.split()
        if len(line) != 9: continue
        vertex = np.vstack(( vertex, np.array([line[0:3]]).astype(float) ))
        factory.insert_vertex(vertex[-1])

    # Grid assamble, exclude elements < min_area
    for line in face_Text:
        line = line.split()
        if len(line)!=5: continue
        A, B, C, _, _ = np.array(line).astype(int)
        side1, side2  = vertex[B-1]-vertex[A-1], vertex[C-1]-vertex[A-1]
        face_area = 0.5*np.linalg.norm(np.cross(side1, side2))
        atotal += face_area
        if face_area > min_area:
            factory.insert_element([A-1, B-1, C-1])
        else:
            xcount += 1.4        # excluded element counter not used
            a_excl += face_area  # total area excluded not used

    grid = factory.finalize()
    
    export_file = os.path.join(pdb_directory , mol_name + '.msh' )
    bempp.api.export(grid=grid, file_name=export_file) 
    
    return grid


Primero se debe completar la información en la frontera, para lo cual si se considera el sistema de ecuaciones diferenciales:

\begin{equation}
\Delta \phi_{1}=\dfrac{1}{\epsilon_{1}}\sum_{k=0}^{N_{q}} \delta \hspace{2pt} q_{k} \hspace{10pt} \text{en} \hspace{10pt} \Omega_{in} \hspace{10pt}  \text{ , } \hspace{10pt} \Delta \phi_{2}=\phi_{2} \hspace{10pt} \text{en} \hspace{10pt} \Omega_{out}  \hspace{10pt}  \text{ y } \hspace{10pt} \Delta \phi_{3}=0 \hspace{10pt} \text{en} \hspace{10pt} \Omega_{stern}
\end{equation}

\begin{eqnarray}
\text{Con} \hspace{10pt} \phi_{1}=\phi_{2} \hspace{10pt} \text{y} \hspace{10pt} \epsilon_{1}\dfrac{\partial \phi_{1}}{\partial \hat{n}} = \epsilon_{2}\dfrac{\partial \phi_{2}}{\partial \hat{n}} \hspace{10pt}  \text{en} \hspace{2pt} \Gamma_{1}  \nonumber \\
\text{Con} \hspace{10pt} \phi_{2}=\phi_{3} \hspace{10pt} \text{y} \hspace{10pt} \epsilon_{2}\dfrac{\partial \phi_{2}}{\partial \hat{n}} = \epsilon_{3}\dfrac{\partial \phi_{3}}{\partial \hat{n}} \hspace{10pt}  \text{en} \hspace{2pt} \Gamma_{2}  \nonumber \\
\end{eqnarray}

Reduciendo estas relaciones con los operadores adecuados, donde $[P]_{A,Dom}^{Range}$ indica al operador $P$, asociado a la ecuación diferencial $A$, al llevar la información del dominio $Dom$ al conjunto de llegada al cual se lleva la información $Range$.
    
\begin{equation}
\begin{bmatrix}
\dfrac{I_{\Gamma_{1}}^{\Gamma_{1}}}{2}+K_{L,\Gamma_{1}}^{\Gamma_{1}} &  -V_{L,\Gamma_{1}}^{\Gamma_{1}} &   0     & 0
\\[2ex]
\dfrac{I_{\Gamma_{1}}^{\Gamma_{1}} }{2}-K_{L,\Gamma_{1}}^{\Gamma_{1}} &  \frac{\epsilon_{1}}{\epsilon_{3}}V_{L,\Gamma_{1}}^{\Gamma_{1}} &  K_{L,\Gamma_{2}}^{\Gamma_{1}} &    -V_{L,\Gamma_{2}}^{\Gamma_{1}}		 
\\[2ex]
-K_{L,\Gamma_{1}}^{\Gamma_{2}} & \frac{\epsilon_{1}}{\epsilon_{3}}V_{L,\Gamma_{1}}^{\Gamma_{2}}   & \dfrac{I_{\Gamma_{2}}^{\Gamma_{2}} }{2}+K_{L,\Gamma_{2}}^{\Gamma_{2}} & -V_{L,\Gamma_{2}}^{\Gamma_{2}}	
\\[2ex]
0 & 0 & \dfrac{I_{\Gamma_{2}}^{\Gamma_{2}} }{2}-K_{Y,\Gamma_{2}}^{\Gamma_{2}} & \frac{\epsilon_{3}}{\epsilon_{2}}V_{Y,\Gamma_{2}}^{\Gamma_{2}}	
\end{bmatrix} 
\left(
\begin{matrix}
\phi_{1} \\[2ex] \dfrac{\partial \phi_{1}}{\partial \hat{n}} \\ 
\phi_{2} \\[2ex] \dfrac{\partial \phi_{2}}{\partial \hat{n}}
\end{matrix}\right)
=
\left(\begin{matrix}
\dfrac{1}{\epsilon_{1}}\sum_{k=0}^{N_{q}} \hspace{2pt} q_{k} \hspace{2pt} G_{L} \\[2ex] 0 \\ 0 \\ 0
\end{matrix}\right)
\end{equation}

Nota: Es posible simplificar $I^{A}_{A}=I_{A}$, la matriz identidad sobre el dominio $A$, ya que este término, sólo aparece cuando se agregan singularidades. Las cuales nacen producto de llevar la información al mismo conjunto de llegada.

In [3]:
def Stern_Solver(mol_name , DPorP , Order , probe_radius = 1.4 , density=3. ,
              stern_thickness=0 , min_area = 5e-3 , plot = False ):
    # Se construye la malla
    mesh = xyzr_to_msh(mol_name , density , probe_radius , stern_thickness , min_area )
    
    # Se abre la malla
    path = os.path.join('Molecule',mol_name,mol_name)
    grid_in = bempp.api.import_grid(path+'.msh')
    grid_out= bempp.api.import_grid(path+'_stern.msh')
    Number_of_in_el  = bempp.api.function_space(grid_in , "DP", 0).global_dof_count
    Number_of_out_el = bempp.api.function_space(grid_out, "DP", 0).global_dof_count
    
    Number_of_el = Number_of_in_el + Number_of_out_el
    print('Number of elements: in= '+str(Number_of_in_el) +' out = '+ 
          str(Number_of_out_el)+' with a total of '+str(Number_of_el)+ '.')
    
    # Se definen los espacios
    dirichl_space_1     = bempp.api.function_space(grid_in ,  DPorP, Order)
    neumann_space_1     = bempp.api.function_space(grid_in ,  DPorP, Order) 
    dual_to_dirichlet_1 = bempp.api.function_space(grid_in ,  DPorP, Order)

    dirichl_space_2     = bempp.api.function_space(grid_out,  DPorP, Order)
    neumann_space_2     = bempp.api.function_space(grid_out,  DPorP, Order) 
    dual_to_dirichlet_2 = bempp.api.function_space(grid_out,  DPorP, Order)
    
    # Se definen los operadores
    identity_1 = sparse.identity( dirichl_space_1, dirichl_space_1, dual_to_dirichlet_1 )
    identity_2 = sparse.identity( dirichl_space_2, dirichl_space_2, dual_to_dirichlet_2 )

    slp_1_1 = laplace.single_layer(neumann_space_1, dirichl_space_1, dual_to_dirichlet_1)
    dlp_1_1 = laplace.double_layer(dirichl_space_1, dirichl_space_1, dual_to_dirichlet_1)

    slp_1_2 = laplace.single_layer(neumann_space_1, dirichl_space_2, dual_to_dirichlet_2)
    dlp_1_2 = laplace.double_layer(dirichl_space_1, dirichl_space_2, dual_to_dirichlet_2)

    slp_2_1 = laplace.single_layer(neumann_space_2, dirichl_space_1, dual_to_dirichlet_1)
    dlp_2_1 = laplace.double_layer(dirichl_space_2, dirichl_space_1, dual_to_dirichlet_1)

    slp_2_2 = modified_helmholtz.single_layer(neumann_space_2, dirichl_space_2, dual_to_dirichlet_2, k)
    dlp_2_2 = modified_helmholtz.double_layer(dirichl_space_2, dirichl_space_2, dual_to_dirichlet_2, k)

    zero_0_2 = bempp.api.ZeroBoundaryOperator(dirichl_space_2, dirichl_space_1, dual_to_dirichlet_1 )
    zero_0_3 = bempp.api.ZeroBoundaryOperator(neumann_space_2, dirichl_space_1, dual_to_dirichlet_1 )

    zero_3_0 = bempp.api.ZeroBoundaryOperator(dirichl_space_1, dirichl_space_2, dual_to_dirichlet_2 )
    zero_3_1 = bempp.api.ZeroBoundaryOperator(neumann_space_1, dirichl_space_2, dual_to_dirichlet_2 ) 
    
    
    # All operators in a given row must have the same range and dual_to_range space.
    # All operators in a given column must have the same domain space.

    # Matrix Assembly
    blocked = bempp.api.BlockedOperator(4, 4)
    blocked[0,0]= 0.5*identity_1 + dlp_1_1 ; blocked[0,1]= -slp_1_1                 
    blocked[1,0]= 0.5*identity_1 - dlp_1_1 ; blocked[1,1]= (ep_in/ep_stern)*slp_1_1 ; blocked[1,2] = dlp_2_1                  ; blocked[1,3] = -slp_2_1
    blocked[2,0]= -dlp_1_2                 ; blocked[2,1]= (ep_in/ep_stern)*slp_1_2 ; blocked[2,2] = 0.5*identity_2 + dlp_2_2 ; blocked[2,3] = -slp_2_2     
    blocked[3,2]= 0.5*identity_2 - dlp_2_2 ; blocked[3,3]= (ep_ex/ep_stern)*slp_2_2 ;
    #print(blocked.shape)

    charged_grid_fun = bempp.api.GridFunction(dirichl_space_1, fun=carga_i)
    zero_grid_fun_1  = bempp.api.GridFunction(neumann_space_1, fun=zero_i )
    zero_grid_fun_2  = bempp.api.GridFunction(dirichl_space_2, fun=zero_i )
    zero_grid_fun_3  = bempp.api.GridFunction(neumann_space_2, fun=zero_i )

    rhs = [charged_grid_fun, zero_grid_fun_1 , zero_grid_fun_2, zero_grid_fun_3]
    
    
   
    # Se resuelve el sistema:
    starting_point = time.time()
    sol, info,it_count = bempp.api.linalg.gmres( blocked, rhs , return_iteration_count=True , use_strong_form=True, tol=1e-3)
    elapsed_time   = round(time.time() - starting_point, 2 )
    print("The linear system was solved in {0} iterations , in {1} seconds".format(it_count, elapsed_time))
    
    solution_dirichl_1, solution_neumann_1, solution_dirichl_2, solution_neumann_2,= sol
    #sln_plot = solution_dirichl_1.plot()

    slp_q = bempp.api.operators.potential.laplace.single_layer(neumann_space_1, x_q.transpose())
    dlp_q = bempp.api.operators.potential.laplace.double_layer(dirichl_space_1, x_q.transpose())
    phi_q = slp_q * solution_neumann_1 - dlp_q * solution_dirichl_1
    
    # total dissolution energy applying constant to get units [kcal/mol]
    total_energy = 2. * np.pi * 332.064 * np.sum(q * phi_q).real
    print("Total solvation energy: {:.2f} [kcal/Mol]".format(total_energy))
    print('----------------------------------')
    
    return Number_of_el,total_energy, elapsed_time
    
    
def carga_i(x, n, domain_index, result):
    global q, x_q, ep_in
    result[:] = np.sum(q/np.linalg.norm( x - x_q, axis=1 ))/(4*np.pi*ep_in)

def zero_i(x, n, domain_index, result):
    result[:] = 0

In [4]:
bempp.api.PLOT_BACKEND = "ipython_notebook"

# We start defining some global variables
ep_in = 4.
ep_ex = 80.
ep_stern = 80.
k     = 0.125
mol_name = 'arg'
stern_thick=2.
path = os.path.join('Molecule',mol_name)


# Now we build .pqr & .xyzr files 
pdb_to_pqr(mol_name='arg', stern_thickness=stern_thick )

# Right side of the Ecn, with the Green Function convolution
q, x_q = np.empty(0), np.empty((0,3))

# Read charges and coordinates from the .pqr file
charges_file = open( os.path.join(path,mol_name+'.pqr'), 'r').read().split('\n')

# Now we extract the charge, position and radii from each Atom
for line in charges_file:
    line = line.split()
    if line[0]!='ATOM': continue
    q = np.append( q, float(line[8]))
    x_q = np.vstack( ( x_q, np.array(line[5:8]).astype(float) ) )    

    # q G_L is made (right side)

%matplotlib inline
sln = Stern_Solver('arg' , 'DP' , 0 ,density=3., stern_thickness=stern_thick)

Global .pqr & .xyzr ready.
Global _stern.pqr & _stern.xyzr ready.
Normal .vert & .face Done
Normal .msh Done
Stern .vert & .face Done
Stern .msh Done
Mesh Ready
Number of elements: in= 943 out = 2547 with a total of 3490.
The linear system was solved in 58 iterations , in 22.97 seconds
Total solvation energy: -66.21 [kcal/Mol]
----------------------------------


In [5]:
bempp.api.set_ipython_notebook_viewer()

ep_in = 4.
ep_ex = 80.
ep_stern = 80.
k     = 0.125
mol_name = 'arg'
path = 'Molecule/'+mol_name+'/'

# Right side of the Ecn, with the Green Function convolution

q, x_q = np.empty(0), np.empty((0,3))

pqr_xyzr = pdb_to_pqr(mol_name , stern_thickness=stern_thick )
charges_file = open(path+mol_name+'.pqr', 'r').read().split('\n')

for line in charges_file:
    line = line.split()
    if line[0]!='ATOM': continue
    q = np.append( q, float(line[8]))
    x_q = np.vstack( ( x_q, np.array(line[5:8]).astype(float) ) )    

text = open(path+'Resultados_stern_30-09.txt','w')

def razon(lista,r):    
    return 1/r**lista

lista = np.arange(0. , 2.001 , 0.25)
prog = razon(2.-lista,2)

max_D = 16.


for esp in ['DP','P']:
    for dens in max_D*prog:
        if esp=='DP':
            Ord=0
            print('Mesh density= {:.2f} [#Ver/A^2] with {}{:.0f}'.format(dens , esp , Ord) )
            sln = Stern_Solver(mol_name , esp , Ord ,density=dens, stern_thickness=stern_thick)
        elif esp=='P':
            Ord=1
            print('Mesh density= {:.2f} [#Ver/A^2] with {}{:.0f}'.format(dens , esp , Ord) )
            sln = Stern_Solver(mol_name , esp , Ord ,density=dens, stern_thickness=stern_thick)
                    
        sln = ' '.join( map(str,sln) )
        inf = ' '.join( map(str, ( esp , Ord , dens) ) )
        text.write(inf + ' ' +sln+'\n')

text.close()

Global .pqr & .xyzr ready.
Global _stern.pqr & _stern.xyzr ready.
Mesh density= 4.00 [#Ver/A^2] with DP0
Normal .vert & .face Done
Normal .msh Done
Stern .vert & .face Done
Stern .msh Done
Mesh Ready
Number of elements: in= 1317 out = 3459 with a total of 4776.
The linear system was solved in 57 iterations , in 31.79 seconds
Total solvation energy: -65.88 [kcal/Mol]
----------------------------------
Mesh density= 4.76 [#Ver/A^2] with DP0
Normal .vert & .face Done
Normal .msh Done
Stern .vert & .face Done
Stern .msh Done
Mesh Ready
Number of elements: in= 1559 out = 4070 with a total of 5629.
The linear system was solved in 56 iterations , in 37.14 seconds
Total solvation energy: -64.83 [kcal/Mol]
----------------------------------
Mesh density= 5.66 [#Ver/A^2] with DP0
Normal .vert & .face Done
Normal .msh Done
Stern .vert & .face Done
Stern .msh Done
Mesh Ready
Number of elements: in= 1867 out = 4780 with a total of 6647.
The linear system was solved in 52 iterations , in 43.18 secon